In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
CITY="FB"

In [ ]:

weather_filename_DD = f"../data/weather/df_Dresden_weather_hourly 2025-03-28_20-51-37.pkl"
weather_filename_FB = f"../data/weather/df_Freiburg_weather_hourly 2025-03-28_20-51-37.pkl"
weather_df_DD = pd.read_pickle(weather_filename_DD)
weather_df_FB = pd.read_pickle(weather_filename_FB)
help_dict_weather = {'DD': weather_df_DD, 'FB': weather_df_FB}

In [ ]:
df_weather = help_dict_weather[CITY]
df_weather.index.name = "datetime_hour"

In [ ]:
datetime = "2025-03-19_10-47-56"
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv"
df_trips_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_trips_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])


help_dict = {'DD': df_trips_DD, 'FB': df_trips_FB}

In [ ]:
weather_df_FB.index.name = "datetime_hour"

In [ ]:
df_trips = help_dict[CITY]

In [ ]:
df_houly_trips= df_trips.drop(columns=["hex_id", "return_count"]).groupby("datetime_hour").agg("sum")

In [ ]:
df_houly_trips

In [ ]:
df = df_houly_trips.copy()

In [ ]:
len(df)

In [ ]:
df_weather.loc[df.index]

In [ ]:
df = pd.concat([df, df_weather.loc[df.index]], axis=1)

In [ ]:
df.rename(columns={"rent_count": "Bike trips"}, inplace=True)

In [ ]:
df_total =  df.reset_index().resample("h", on="datetime_hour").agg({"Temperature": "mean", "Humidity": "mean", "Wind": "mean", "Precipitation": "sum", "Bike trips": "sum"})

In [ ]:
df_total['Rain']= df_total['Precipitation'].apply(lambda x: True if x > 0 else False)

# weekday, month

In [ ]:
df_total

In [ ]:
cols =  [col for col in df_total.columns if col != "Bike trips"]+["Bike trips"] 


df_total = df_total[cols]

In [ ]:
corr = df_total.corr()

In [ ]:
import numpy as np

In [ ]:
# # Generate a mask for the upper triangle; True = do NOT show
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(6, 5))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
# More details at https://seaborn.pydata.org/generated/seaborn.heatmap.html
sns.heatmap(
    corr,          # The data to plot
    mask=mask,     # Mask some cells
    cmap=cmap,     # What colors to plot the heatmap as
    annot=True,    # Should the values be plotted in the cells?
    vmax=.5,       # The maximum value of the legend. All higher vals will be same color
    vmin=-.5,      # The minimum value of the legend. All lower vals will be same color
    center=0,      # The center value of the legend. With divergent cmap, where white is
    square=True,   # Force cells to be square
    linewidths=.5, # Width of lines that divide cells
    cbar_kws={"shrink": .5}  # Extra kwargs for the legend; in this case, shrink by 50%
)
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/weather_corplot_{CITY}.png', format='png', bbox_inches='tight')
plt.show()


In [ ]:
df_total.Rain.value_counts()

In [ ]:
df_total_d =  df_total.reset_index().resample("4h", on="datetime_hour").agg({"Temperature": "mean", "Humidity": "mean", "Wind": "mean", "Precipitation": "sum", "Bike trips": "sum"})

In [ ]:
df_total_d

In [ ]:
df_total_d['Rain']= df_total_d['Precipitation'].apply(lambda x: True if x > 0 else False)

In [ ]:
f, ax = plt.subplots(figsize=(6, 5))
sns.lmplot(data=df_total_d, x="Temperature", y="Bike trips", hue="Rain", height=5, aspect=2)
plt.show()

In [ ]:
plt.figure(figsize=(6, 5))
sns.reset_defaults()
sns.set_style("white")
sns.set_palette("husl")

sns.pairplot(data=df_total_d, hue="Rain", palette={True: "cornflowerblue", False: "firebrick"}, plot_kws={'s': 10,'alpha': 0.5}, height=2)
plt.xlabel('Temperature')
plt.ylabel('Number of trips per hour')


plt.grid(True) 
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/weather_pairplot_{CITY}.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
# Create bins
df_total["temp_bin"] = pd.cut(df_total["Temperature"], bins=5, precision=0)
df_total["humid_bin"] = pd.cut(df_total["Humidity"], bins=5, precision=1)

# Pivot table
heatmap_data = df_total.pivot_table(index="humid_bin", columns="temp_bin", values="Bike trips", aggfunc="mean")

In [ ]:
heatmap_data

In [ ]:
heatmap_data = heatmap_data.sort_values(by="precip_bin", ascending=False)

In [ ]:
heatmap_data

In [ ]:
plt.figure(figsize=(6, 5))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(heatmap_data, cmap=cmap, annot=True, fmt=".0f")
plt.xlabel("Temperature")
plt.ylabel("Humidity")
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/precip_temp_heatmap_{CITY}.png', format='png', bbox_inches='tight')
plt.show()
